In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from threeML import *
import numpy as np
from threeML.plugins import *
from skylab.datasets import Datasets
from astropy import units as u

import os, sys, glob, abc

def read(filelist):
    data = []
    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])
    return data

# Where is the dataset stored?
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
exp = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"
mc = read(glob.glob(sim_files))

# Set the angular error floor to 0.2 degrees
#exp['angErr'][exp['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
#mc['angErr'][mc['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)

exp['ra']=np.random.uniform(0,2*np.pi,size=len(exp))
grl = np.load("/data/i3store/users/mjlarson/student_data/GRL/IC86_2012_exp.npy")
livetime = np.sum(grl['livetime'])



Bad key "text.kerning_factor" on line 4 in
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/data/disk01/home/jasonfan/threeML/threeML/__init__.py:12: UserWarning: No DISPLAY variable set. Using backend for graphics without display (Agg)
  warnings.warn("No DISPLAY variable set. Using backend for graphics without display (Agg)")
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of K to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/paramet

In [2]:
from imp import reload
reload(IceCubeLike)

import warnings
warnings.filterwarnings("ignore")
#data = np.random.choice(exp, 10000).copy()
#jl._data_list['test'].llh_model.update_data(data)
IceCube=IceCubeLike.IceCubeLike("test",exp,mc,exp,verbose=True,background_time_profile= (56224,56324),sampling_width=np.radians(1))
del mc

In [3]:
source1_sp = Powerlaw()
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_sp.K=1e-13
source1_neutrino = Powerlaw()
source1_neutrino._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_neutrino.piv=1000
source1_neutrino.K=1e-13
source1_neutrino.index=-2
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1 = PointSource("source1", ra=83.63, dec=22.0145, spectral_shape=source1_sp)
source1_nu = IceCubeLike.NeutrinoPointSource("source1_nu",ra=83.63, dec=22.0145,spectral_shape=source1_neutrino,energy_unit=u.GeV)

model = Model(source1,source1_nu)
model.source1.spectrum.main.Powerlaw.K.fix = True
model.source1.spectrum.main.Powerlaw.index.fix = True
model.source1_nu.spectrum.main.Powerlaw.K.bounds = (1e-30, 1e-5)
model.source1_nu.spectrum.main.Powerlaw.index.bounds = (-4, -1)
#IceCube.set_model(model)
IceCubedata = DataList(IceCube)
jl = JointLikelihood(model, IceCubedata)

In [4]:
test=IceCubeLike.sensitivity(jl)

In [ ]:
test.set_backround(grl,model ,time_window=100,start_time=56224)

In [ ]:
import mla 
basespectrum = mla.spectral.PowerLaw(1e3 , 1 , -2)
test.set_injection(background_time_profile = (56224,56324))

In [ ]:
import numpy.lib.recfunctions as rf
basespectrum = mla.spectral.PowerLaw(1e3 , 1 , -2)
basespectrum.A=1e-13
test.PS_injector.update_spectrum(basespectrum)
#test.jl_value.llh_model.update_spectrum(basespectrum)
testdata=test.draw_data()
sample=test.draw_signal()
sample = rf.drop_fields(sample, [n for n in sample.dtype.names \
                       if not n in testdata.dtype.names])
testdata=np.concatenate([testdata,sample])
#test.jl_value.llh_model.update_data(test.draw_data())
#test.jl_value.llh_model.modify_injection(test.draw_signal())
#np.save("test3.npy",testdata)
#testdata=np.load("test.npy")
test.jl_value.llh_model.update_data(testdata)

In [ ]:
(test.jl_value.llh_model.data["run"]>1000000).sum()

In [ ]:
test.jl_value.verbose=True
test.jl.fit()

In [ ]:
print(test.jl_value.llh_model.data["run"]>1000000).sum()

In [ ]:
K=np.linspace(5e-14,1e-12,50)
gamma=np.linspace(-1,-4,30)
ns=np.linspace(0,60,50)
result=[]

In [ ]:
for j in gamma:
    source1_neutrino = Powerlaw()
    source1_neutrino._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
    source1_neutrino.piv=1000
    source1_neutrino.index=j
    source1_nu = IceCubeLike.NeutrinoPointSource("source1_nu",ra=83.63, dec=22.0145,spectral_shape=source1_neutrino)
    model = Model(source1_nu)
    test.jl_value.set_model(model)
    test.jl_value.update_model()
    for i in K:
        source1_neutrino.K=i
        ns = (test.jl_value.llh_model.sim['ow'] * source1_neutrino(test.jl_value.llh_model.sim['trueE']) * test.jl_value.llh_model.signal_time_profile.effective_exposure() *24*3600).sum() 
        temp=test.jl_value.llh_model.eval_llh_ns(ns)
        result.append([i,j,temp[0],temp[1]])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
result=np.array(result)
x=result[:,0]
y=result[:,1]
z=result[:,3]

## Equivalently, we could do that all in one line with:
# x,y,z = np.genfromtxt('eye_.txt', delimiter=',', usecols=(0,1,2))
z[z<0]=0
x=np.unique(x)
y=np.unique(y)
X,Y = np.meshgrid(x,y)

Z=z.reshape(len(y),len(x))
index=result[np.argmax(result[:,3])]
fig, ax = plt.subplots(figsize = (6,5))
plt.pcolormesh(X,np.flip(Y),Z,vmin=380, vmax=450)
plt.colorbar()
plt.scatter(index[0],index[1],label="Best fit")
plt.title("Spectral index vs Flux Norm",fontsize=14)
plt.xlabel(r"Flux Norm($GeV^{-1} s^{-1} cm^{-2}$)",fontsize=14)
plt.ylabel("Spectral index",fontsize=14)
plt.legend()
plt.show()

In [ ]:
from imp import reload
import scipy.optimize
reload(mla.core)

sp= PowerLaw(1000,1e-15,2)
testdata=np.load("test3.npy")
testdata['time']=0.5
import warnings
warnings.filterwarnings("ignore")
test=mla.core.LLH_point_source(np.radians(83.63) ,np.radians(22.0145) , testdata , sim , "PowerLaw", background = data , \
                              )

In [ ]:
gamma=np.linspace(-1.5,-3.5,30)
ns=np.linspace(10,70,50)
result=[]
for i in gamma:
    test.update_energy_weight(gamma=i)
    for j in ns:
        temp=test.eval_llh_ns(j)
        result.append([i,j,temp[0],temp[1]])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
result=np.array(result)
x=result[:,0]
y=result[:,1]
z=result[:,3]

## Equivalently, we could do that all in one line with:
# x,y,z = np.genfromtxt('eye_.txt', delimiter=',', usecols=(0,1,2))
z[z<0]=0
x=np.unique(x)
y=np.unique(y)
X,Y = np.meshgrid(x,y)

Z=z.reshape(len(y),len(x))
index=result[np.argmax(result[:,3])]
fig, ax = plt.subplots(figsize = (6,5))
plt.pcolormesh(X,np.flip(Y),Z)
plt.colorbar()
plt.scatter(index[0],index[1],label="Best fit")
plt.scatter(test.get_fit_result()[0],test.get_fit_result()[1],label="Best fit")
plt.title("ns vs Spectral index",fontsize=14)
plt.xlabel(r"Spectral index",fontsize=14)
plt.ylabel("ns",fontsize=14)
plt.legend()
plt.show()